In [1]:
import numpy as np
from scipy import sparse
from tables import *
import time
from tqdm import tqdm_notebook as tqdm

data_path = "/mnt/disks/sec/data/"

In [48]:
class MappedCit(IsDescription):  
    cite = UInt32Col()
    citant = UInt32Col()
    cite_array = UInt32Col()
    citant_array = UInt32Col()


In [2]:
h5file = open_file(data_path+"citations.h5", mode="a", title="citations")



In [5]:
table_cits = h5file.root.citations.cits

In [50]:
nb_items = int(1.5e+09)
citants = np.full(nb_items,np.nan)
cites_IDs = set()

In [51]:
idx = 0
for row in table_cits:
    citants[idx] = int(row['citant'])
    cites_IDs.add(int(row['cite']))
    idx += 1

In [52]:
citants = citants.astype(np.int)
citants = citants[np.where(~np.isnan(citants))[0]]
citants_IDs = np.unique(citants)

In [53]:
nb_l = len(citants)

In [54]:
cites_IDs = np.array(list(cites_IDs))
all_IDs = np.unique(np.concatenate([cites_IDs,citants_IDs]))

In [55]:
nb_IDs = len(all_IDs)
nb_citants = len(citants_IDs)
nb_cites = len(cites_IDs)

In [63]:
table_map = h5file.create_table(h5file.root.citations, 'mappedCits2', MappedCit)
citation = table_map.row

In [58]:
all_IDs_dict = {}
for i in range(nb_IDs):
    all_IDs_dict[all_IDs[i]] = i

In [ ]:
for i in tqdm(range(nb_l)):
    row = table_cits[i]
    citation['cite'] = row['cite']
    citation['citant'] = row['citant']
    #citation['citant_array'] = np.where(all_IDs==row['citant'])[0]
    #citation['cite_array'] = np.where(all_IDs==row['cite'])[0]
    citation['citant_array'] = all_IDs_dict[row['citant']]
    citation['cite_array'] = all_IDs_dict[row['cite']]
    citation.append()
table_map.flush()


/home/User1/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:1: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  """Entry point for launching an IPython kernel.


A Jupyter Widget

In [3]:
table_map = h5file.root.citations.mappedCits2

NoSuchNodeError: group ``/citations`` does not have a child named ``mappedCits2``

In [67]:
cit_mat = sparse.lil_matrix((nb_IDs,nb_IDs))

In [ ]:
for i in tqdm(range(nb_l)):
    row = table_map[i]
    cit_mat[row['cite_array'],row['citant_array']] = 1 

/home/User1/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:1: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  """Entry point for launching an IPython kernel.


A Jupyter Widget

In [ ]:
sparse.save_npz(data_path+'cit_mat.npz',cit_mat)
np.save(data_path+'all_IDs.npy',all_IDs)

In [9]:
dict_cits = {}

In [12]:
nb_l = len(table_cits)


In [13]:
for i in tqdm(range(nb_l)):
    row = table_cits[i]
    cite = row['cite']
    citant = row['citant']
    if citant in dict_cits:
        dict_cits[citant] = np.append(dict_cits[citant],cite)
    else:
        dict_cits[citant] = np.array([cite],dtype=np.uint32)

/home/User1/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:1: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  """Entry point for launching an IPython kernel.


A Jupyter Widget

In [15]:
class DictCits(IsDescription):
    citant = UInt32Col()
    cites = StringCol(itemsize=4000, dflt=" ", pos=0)


In [16]:
h5file.close()

In [50]:
h5file = open_file('/mnt/disks/sdb/data/' + 'DictCits.h5', mode="w", title="DictCits")
group = h5file.create_group("/", 'pub_exp')
table = h5file.create_table(group, 'DictCits', DictCits)
citations = table.row


In [18]:
nb_citants = len(dict_cits)

In [23]:
keys_cits = list(dict_cits.keys())

In [51]:
for i in tqdm(range(nb_citants)):
    citant = keys_cits[i]
    citations['cites'] = dict_cits[citant].tostring()
    citations['citant'] = citant
    citations.append()
table.flush()

/home/User1/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:1: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  """Entry point for launching an IPython kernel.


A Jupyter Widget

In [49]:
h5file.close()

In [29]:
np.fromstring(dict_cits[keys_cits[0]].tostring(),dtype=np.int32)

array([ 3069030,  5032476,  8445497,  8905471, 11375513, 11412217,
       11558363, 12050856, 12439677, 13614399, 14687146, 15345523,
       15622846, 15838651, 16231099, 17076611, 17204359, 17428859,
       17902646, 18615012, 18921491, 18970109, 19384578, 19346762,
       19456907, 19781630, 19983693, 20164357, 22042363, 22276655,
       22573948, 22606829, 23053309, 23214880, 23585038, 23673471,
       24252976, 24287932, 24634231, 24731642, 24753174, 24783052,
       25043518, 25060098, 25145671, 25180629, 25180301, 25551611,
       25637514, 25749119, 25991104, 26009064, 26148562, 26475745,
       26408364, 26484714, 26550040, 26550041, 26568914, 26624635,
       26671888, 27090695, 27197176, 27183294, 27305808, 27420785,
       27711609, 28092252, 28125480, 28442671, 28667026, 28682218,
       28686628, 28707652, 28667916, 28740594, 29071228, 29477824,
       29354001, 29584053, 29575284, 29650252, 29721468, 29711431,
       29914275, 30110176, 30304361, 30278229, 30946965, 31023